In [21]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [2]:
def pull_stats(year):   

    # 1
    url = "https://www.basketball-reference.com/leagues/NBA_{}_per_game.html".format(year)
    soup = BeautifulSoup(urlopen(url))
    
    #2
    headers = [th.getText() for th in soup.findAll('tr')[0].findAll('th')]

    #3
    headers = headers[1:]

    #4
    rows = soup.findAll('tr')[1:]
    
    #5
    player_stats = [[td.getText() for td in rows[i].findAll('td')]
                for i in range(len(rows))]
    
    #6
    stats = pd.DataFrame(player_stats, columns = headers)
    
    #7
    stats['Year'] = year
    
    return stats

In [3]:
def pull_salary(year, pages=50):
    #1
    full = []
    
    #2
    for i in range(1,pages+1):
        
        #3
        url = "http://www.espn.com/nba/salaries/_/year/{}/page/{}".format(year, i)

        soup = BeautifulSoup(urlopen(url))

        headers = [td.getText() for td in soup.findAll('tr')[0].findAll('td')]

        rows = soup.findAll('tr')[1:]

        player_salary = [[td.getText() for td in rows[i].findAll('td')]
                    for i in range(len(rows))]
        
        salary = pd.DataFrame(player_salary, columns = headers)
        salary['Year'] = year
        
        #4
        for i in range(len(salary)):
            salary['NAME'][i] = salary['NAME'][i].split(',')[0]
        
        full.append(salary)
    
    #5
    full = pd.concat(full)
    
    #6
    full.drop(full[full['RK']=='RK'].index, inplace=True)
    
    #7
    full = full.reset_index().drop(columns='index')
    
    return full

In [31]:
def create_df(years):
    
    #1
    final = pd.DataFrame()
    
    #2
    for year in years:
        temp_sta = pull_stats(year)
        temp_sal = pull_salary(year)
        
        #3
        cols = list(temp_sta.columns)
        cols.append('Salary')
        new = pd.DataFrame(columns=cols)
        
        #4
        for i in range(len(temp_sta)):
            player = temp_sta['Player'][i]
            index = temp_sal[temp_sal['NAME']==player].index.values
            
            #5
            if index.size == 0:
                continue
            else:
                new = new.append(temp_sta.iloc[i])
                new['Salary'][i] = temp_sal['SALARY'][index].values[0]
        
        #6
        final = final.append(new)
    
    #7
    final.replace('', np.nan, inplace=True)
    final.dropna(inplace=True)
    final = final.loc[(final['Tm'] == 'TOT')|~final['Player'].duplicated()]
    
    return final

In [32]:
test = create_df([2018, 2019])

C:\Users\ande5\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [33]:
test.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year,Salary
2,Steven Adams,C,24,OKC,76,76,32.7,5.9,9.4,.629,...,4.0,9.0,1.2,1.2,1.0,1.7,2.8,13.9,2018,"$22,471,910"
3,Bam Adebayo,C,20,MIA,69,19,19.8,2.5,4.9,.512,...,3.8,5.5,1.5,0.5,0.6,1.0,2.0,6.9,2018,"$2,955,840"
6,LaMarcus Aldridge,C,32,SAS,75,75,33.5,9.2,18.0,.510,...,5.2,8.5,2.0,0.6,1.2,1.5,2.1,23.1,2018,"$21,461,010"
7,Jarrett Allen,C,19,BRK,72,31,20.0,3.3,5.5,.589,...,3.4,5.4,0.7,0.4,1.2,1.1,2.0,8.2,2018,"$2,034,120"
9,Tony Allen,SF,36,NOP,22,0,12.4,2.0,4.1,.484,...,1.2,2.1,0.4,0.5,0.1,0.9,2.2,4.7,2018,"$1,471,382"


In [34]:
test.to_csv('Final.csv', index = False)

In [35]:
len(test)

513